Imports

In [10]:
import pandas as pd
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import folium
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe
import numpy as np
# import k-means from clustering stage
from sklearn.cluster import KMeans
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

Get the table

In [11]:
link = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df = pd.read_html(link)[0]
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Take out boroughs not assigned and replace not assigned neighbourhoods

In [12]:
df.drop(df[df.Borough == "Not assigned"].index, inplace=True)
df.Neighbourhood.replace("Not assigned", df.Borough, inplace=True)
df.reset_index(inplace=True)
df.drop("index", axis=1, inplace=True)
df.head()

#df.reset_index(inplace=True)
#df.drop("index", axis=1, inplace=True)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,Harbourfront
3,M5A,Downtown Toronto,Regent Park
4,M6A,North York,Lawrence Heights


Join repeated postcodes

In [13]:
groups_postcodes = df.groupby("Postcode")
#groups_postcodes.groups

In [14]:
#inds = groups_postcodes.get_group("M5A")["Neighbourhood"].index
#a = df.iloc[inds].Neighbourhood
for key in groups_postcodes.groups.keys():
    inds = groups_postcodes.get_group(key)["Neighbourhood"].index
    serie = df.iloc[inds].Neighbourhood
    #print(serie)
    string = serie.str.cat(sep=", ")
    #print(string)
    df.iloc[inds, 2] = string

# drop_duplicates
df.drop_duplicates(subset="Postcode", keep='first', inplace=True)

In [16]:
df.reset_index(inplace=True)
df.drop("index", axis=1, inplace=True)

In [18]:
df.head(12)

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Rouge, Malvern"
7,M3B,North York,Don Mills North
8,M4B,East York,"Woodbine Gardens, Parkview Hill"
9,M5B,Downtown Toronto,"Ryerson, Garden District"
